This notebook example involves using a managed version of Trino (Starburst). It will work without Trino provided you are able to import data into a Trino cluster connected to a lake. We will be using one month of Yellow Taxi dataset from https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page and a zone look up file provided on the same page. Please download both files and register in either your Starburst or Trino cluster before proceeding.

In [ ]:
import ibis
import pandas as pd
ibis.options.interactive = True

#from trino.auth import OAuth2Authentication

IMPORTANT!!!! Change your user, host, port, database, schema and roles to be relevant to your Starburst Galaxy setup. If you are using OAuth2, uncomment the keyword lines roles, and auth. Then comment PASSWORD to proceed. 

In [ ]:
import os

In [ ]:
con = ibis.trino.connect(
  user=os.environ['user'],
  host=os.environ['host'],
  password=os.environ['password'],
  port=443,
  database=os.environ['database'],
  schema=os.environ['schema'],
  #roles="accountadmin",
  #auth=OAuth2Authentication(),
  http_scheme="https"
)

Within ibis con.list_tables() allows us to list all the tables.


In [ ]:
con.list_tables()

Ibis tables in trino can be stored through the use of con.table. We're going to create two ibis tables from our tables below:

In [ ]:
nycjantrips = con.table("taxizonenyc")
zonelookup = con.table("zonelookup")

In ibis we can check the schema of the tables we just store through .schema()

In [ ]:
nycjantrips.schema()

In [ ]:
zonelookup.schema()

We're going to preview the dataset with ibis slice method. We can see the first 10 rows here. We also included ibis.options.interactive = True
 at the start of our notebook which allows us to display the ibis tables in a prettified way.

In [ ]:
nycjantrips[0:10]

To understand the dataset a little more we can try an order by. Looks like there are some columns with passenger count of undefined. In this case we're going to want
to curate the dataset and clean it up a bit to ensure more accurate data.

In [ ]:
nycjantrips.order_by(nycjantrips.trip_distance.desc())

We can chain together expressions with filter - similar to a WHERE clause in SQL. We can see nan (not a number) involved, ibis also has built-in support for that.

In [ ]:
nyc_filtered = nycjantrips.filter((nycjantrips.passenger_count != 0) | (nycjantrips.passenger_count.isnan() is False))
nyc_filtered

You can see with the command below that nan has been filtered out! 

In [ ]:
nyc_filtered.order_by(nyc_filtered.trip_distance.desc())

Let's add a column to our dataset. I want to add a column to help calculate the average ride duration. We are going to use the Ibis 'Delta' function for this result
Ibis is also pretty cool and create a column in isolation:

In [ ]:
ride_duration = nyc_filtered.tpep_dropoff_datetime.delta(nyc_filtered.tpep_pickup_datetime, "minute").name("rideminutes")
ride_duration

We can also combine the column with our original table using the 'mutate' method shown here. 

In [ ]:
nycjanduration = nyc_filtered.mutate(rideminutes=nyc_filtered.tpep_dropoff_datetime.delta(nyc_filtered.tpep_pickup_datetime, "minute"))
nycjanduration["vendorid","rideminutes","trip_distance"]

In [ ]:
nycjanduration["vendorid","rideminutes","trip_distance"].head(3)

Next up are some basic analytics and aggretions in Ibis  - let's get total revenue with sum(), longest trip with max(), and average trip duration with mean(). 
Ibis is able to chain expressions similar to pandas. 

In [ ]:
#some basic analytics - let's get total revenue, longest trip. 
insights = nycjanduration.agg(
    [
        ibis._.count().name("total_trips"),
        ibis._["total_amount"].sum().name("total_revenue"),
        ibis._["trip_distance"].sum().name("total_distance_all"),
        ibis._["rideminutes"].max().name("longest"),    
        ibis._["rideminutes"].mean().round(2).name("average_ride")
    ]
)
insights

Wait, the longest trip seems a bit... lengthy... Note: we added a .round function to display the average ride more nicely. Let's check out the ride itself. 

In [ ]:
nycjanduration.filter(nycjanduration["rideminutes"] == 10030)

7 day trip? looks like the trip distance is zero, we can decide to remove the row from future calculations of average
Let's remove the outliers and join with a lookup table to get more information about the "where" of our analytical datasets - zones.

In [ ]:
nycjanduration_new = (
    nycjanduration.filter(nycjanduration.trip_distance != 0.0)
)
nycjanduration_new
    

Let's create a cleaner set similar to before.

In [ ]:
insights_new = nycjanduration_new.agg(
    [
        ibis._.count().name("total_trips"),
        ibis._["total_amount"].sum().name("total_revenue"),
        ibis._["trip_distance"].sum().name("total_distance_all"),
        ibis._["rideminutes"].max().name("longest"),    
        ibis._["rideminutes"].mean().round(2).name("average_ride")
    ]
)
insights_new

You can already see a slightly more massaged dataset - the longest trip is lower, alongside average_ride has changed and the total number of trips has gone down by almost 40k

Next up we want to do something more powerful - join with related datasets to get more insights on geographical behaviour of taxi trips around NYC. Let's look over the zonelookup table again.

In [ ]:
zonelookup

We can see pulocationid is int64, so we must cast to have the tables fully joined. Ibis supports casting data types within its library as well. In the line below, we use .cast("str") to ensure the two tables can be joined together. You can try without the cast and see what happens :). 


In [ ]:
joineddata = nycjanduration_new.inner_join(zonelookup, nycjanduration_new.pulocationid.cast("str") == zonelookup.locationid)

In [ ]:
joineddata

Now we can do more cool things in ibis with group bys and aggregate by with zones and boroughs!


In [ ]:
groupbyboroughtrips = (
    joineddata
    .group_by("zone")
    .aggregate(
        trips=joineddata.vendorid.count(),
        totalrev=joineddata.fare_amount.sum(),
        totalpassengers=joineddata.passenger_count.sum(),
        averageride=joineddata.rideminutes.mean().round(2)
        
        )
    .order_by(ibis.desc("totalrev"))
    .limit(10)
)
groupbyboroughtrips
    

If you want to see what sql ibis generates, you can use the ibis.to_sql() method.

In [ ]:
ibis.to_sql(groupbyboroughtrips)

Airport rides give the most revenue to taxi companies, that makes a lot of sense. What about by week? Any change in behaviour over the month? 
P.S This ibis code below at the time of writing uses a pre-release function "bucket". This is included with future version of IBIS. When bucket is released, you can uncomment the lines below for a temporal grouping. If you recieve an error 
the function does not exist, you can uncomment the line below to try to upgrade to a pre-release version of ibis:

In [ ]:
#python -m pip install -U 'ibis-framework[trino]' --pre

In [ ]:
'''groupbyweek = (
    joineddata
    .group_by(joineddata.tpep_pickup_datetime.bucket(weeks=1).name("weeks"))
    .aggregate(
        trips=joineddata.vendorid.count(),
        totalrev=joineddata.fare_amount.sum(),
        totalpassengers=joineddata.passenger_count.sum(),
        averageride=joineddata.rideminutes.mean().round(2)

        )
    .order_by(ibis.desc("totalrev"))
    .limit(10)
)
groupbyweek'''

Let's write our result tables back to trino (to show some write functionality, of course).


In [ ]:
con.create_table("groupbyboroughtrips", groupbyboroughtrips)

There you have it, a quick tutorial with Ibis, and Starburst Galaxy! 